**Installing the packages**

In [1]:
#!pip install simpy
import random
import simpy as sy

**Intializing the variable and creating the simulation**

In [2]:
#Declaring the variables

num_checkers = 10  # Number of Checkers 
num_scanners = 5   #Number of scanners 
wait_time = 0      #Initial Waiting Time set to 0
total_pax = 1      #Total number of passengers initialized to 1
num_pax = 100      #Overall Passengers set to 100
runtime =  500     #End simulation when runtime crosses 500 mins

arrival_rate = 50  #To simulate a busy airport
check_rate = 0.75  #As mentioned in the 

class Airport(object):
    def __init__(self, env, num_checkers, num_scanners):
        self.env = env
        self.checker = sy.Resource(env, num_checkers)  #Number of boarding pass checkers 
        self.scanners = []
        for i in range(0, num_scanners):              #Number of scanners 
            self.scanners.append(sy.Resource(env))
            
    def BP_check(self, pax):
        service_time = random.expovariate(1/check_rate)
        yield self.env.timeout(service_time)
        
    def scan(self, pax):
        scan_time = random.uniform(0.5, 1)
        yield self.env.timeout(scan_time)
        
    def Passenger(self, env, number):
        global wait_time #global average wait time
        global total_pax  
        arrival_time = env.now
        scan_queue = []  #Every scanner has its own queue
        
        with self.checker.request() as request:
            yield request
            yield env.process(self.BP_check(number))
            
        for scanner in self.scanners:
            scan_queue.append(len(scanner.queue))  #getting the length of each scanner 
            
        #Find the shortest scanner queue
        min_index = min(scan_queue)
        short_queue_index = scan_queue.index(min_index)
        
        with self.scanners[short_queue_index].request() as request:
            yield request
            yield env.process(self.scan(number))
            
        exit_time = env.now
        wait_time += (exit_time - arrival_time)
        total_pax += 1

    def setup(self, env, num_pax):
        yield env.timeout(random.expovariate(arrival_rate)) 
        env.process(self.Passenger(env, num_pax))
        
        
#Running the simulation
env = sy.Environment()
api = Airport(env, num_checkers, num_scanners)

for i in range(0,num_pax):
    env.process(api.setup(env, i))

env.run(until = runtime)
avg_wait_time = wait_time / total_pax

print("Avg waiting time is %f"  %avg_wait_time)


Avg waiting time is 8.104722
